In [70]:
import pandas as pd
import plotly.express as px
import bamboolib as bam
import requests as req
import yfinance as yf
from yahoofinancials import YahooFinancials
import json
import os as os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [71]:
res = req.get("https://api.alternative.me/fng/?limit=0&format=csv")
os.system("rm data/fng.json")
os.system("rm data/fng.csv")
resfile = open("data/fng.json","w")
resfile.writelines(str(res.text))

rm: cannot remove 'data/fng.json': No such file or directory


In [72]:
os.system("sed -i '1,3d' data/fng.json")
for i in range(0,5):
    os.system("sed -i '$d' data/fng.json")
os.system("mv data/fng.json data/fng.csv")

0

In [81]:
fng = pd.read_csv("data/fng.csv")
# Column order messed up, renaming to fix it
fng = fng.rename(columns={'fng_value': 'date', 'fng_classification': 'value', 'date': 'fng_class'})
# Step: Change data type of date to Datetime
fng['date'] = pd.to_datetime(fng['date'], format='%d-%m-%Y')
fng["sma"]=fng["value"].rolling(30).mean()

In [82]:
fng = fng.sort_values(by=['date'], ascending=[True])
btcusd = yf.download('BTC-USD', 
                     period="d",
                      start=str(fng["date"].iloc[0]).split(" ")[0], 
                      end=str(fng["date"].iloc[len(fng)-1]).split(" ")[0], 
                      progress=False)
# btcusd= btcusd.set_index("Date")
fng = fng.set_index("date")
fng = pd.concat([btcusd, fng], axis=1, join="inner")
fng = fng.reset_index()
# Step: Rename column
fng = fng.rename(columns={'index': 'date'})

In [75]:
fig = px.line(fng.sort_values(by=['date'], ascending=[True]), x='date', y='Close')
fig.update_yaxes(type='log', tickformat='.1e')
fig

In [76]:
fig = px.line(fng.sort_values(by=['date'], ascending=[True]).dropna(subset=['sma']), x='date', y='sma')
fig

In [77]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=fng['date'],y=fng['sma'],name="Fear and Greed sma"),secondary_y=False)
fig.add_trace(go.Line(x=fng['date'],y=fng['Close'],name="Daily Close"),secondary_y=True)
fig.update_annotations()
fig.show()

In [83]:
risk = fng.copy() 
risk["Change"]= risk['Close'].pct_change(periods=1)
#risk["rsi"]=risk["Change"].rolling(15).mean()
risk["Change-direction"] = risk["Change"].apply(lambda x: "up" if x >=0 else "down")
risk["Change-abs"]= risk['Change'].abs()
# Step: Pivot dataframe from long to wide format using the variable column 'Change-direction' and the value column 'Change'
risk = risk.set_index(['date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'value', 'fng_class', 'sma','Change', 'Change-direction'])['Change-abs'].unstack(-1).reset_index()
risk.columns.name = ''
risk= risk.fillna(0)
risk['down']=risk['down'].rolling(15).mean()
risk['up']=risk['up'].rolling(15).mean()
risk['RSI']=100-(100/(1+(risk['up']/risk['down'])))

In [79]:
risk["RSI-sma"]= risk["RSI"].rolling(60).mean()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=risk['date'],y=risk['RSI-sma'],name="Sma of RSI"),secondary_y=False)
fig.add_trace(go.Line(x=risk['date'],y=risk['Close'],name="Daily close"),secondary_y=True)
fig.show()

In [80]:
risk["risk"]=(risk["RSI-sma"]+risk["sma"])/2
risk
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=risk['date'],y=risk['risk'],name="Risk"),secondary_y=False)
fig.add_trace(go.Line(x=risk['date'],y=risk['Close'],name="Daily close"),secondary_y=True)
fig.update_yaxes(type="log",secondary_y=True)
fig.show()